In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from numpy import sin, cos, tan
from scipy import integrate
from quadcopter_animation import animation

In [ ]:
model = torch.load('neural_networks/HOVER_TO_HOVER_NOMINAL_.pt')
model.eval()

for p in model.parameters():
    p.requires_grad = False

In [ ]:
def neural_network(X, Mx, My, Mz):
    with torch.no_grad():
        X = list(X[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]])+[Mx,My,Mz]
        u = model(torch.tensor(X, dtype=torch.float32)).detach().numpy()
        return u

neural_network(np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))

In [ ]:
def transform(X):
    # TRANSFORM COORDINATES TO BODY FRAME
    x, y, z, vx, vy, vz, phi, theta, psi, p, q, r, utau1, utau2, utau3, utau4 = X
    Rx = np.array([[1, 0, 0], [0, np.cos(phi), -np.sin(phi)], [0, np.sin(phi), np.cos(phi)]])
    Ry = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    Rz = np.array([[np.cos(psi), -np.sin(psi), 0], [np.sin(psi), np.cos(psi), 0], [0, 0, 1]])
    R = Rz@Ry@Rx
    
    # new state variables
    xn, yn, zn = -R.T@[x,y,z]
    vxn, vyn, vzn = R.T@[vx,vy,vz]
    
    Xn = np.array([xn, yn, zn, vxn, vyn, vzn, phi, theta, psi, p, q, r, utau1, utau2, utau3, utau4])
    return Xn

def transform_back(X):
    # TRANSFORM COORDINATES TO WORLD FRAME
    x, y, z, vx, vy, vz, phi, theta, psi, p, q, r, utau1, utau2, utau3, utau4 = X
    Rx = np.array([[1, 0, 0], [0, np.cos(phi), -np.sin(phi)], [0, np.sin(phi), np.cos(phi)]])
    Ry = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    Rz = np.array([[np.cos(psi), -np.sin(psi), 0], [np.sin(psi), np.cos(psi), 0], [0, 0, 1]])
    R = Rz@Ry@Rx
    
    # new state variables
    xn, yn, zn = -R@[x,y,z]
    vxn, vyn, vzn = R@[vx,vy,vz]
    
    Xn = np.array([xn, yn, zn, vxn, vyn, vzn, phi, theta, psi, p, q, r, utau1, utau2, utau3, utau4])
    return Xn

In [ ]:
# trajectory dataset
dataset_path = 'datasets/free_final_velocity_tau=0.06_no_moments.npz'
dataset = np.load(dataset_path)

g         = dataset['g']
mass      = dataset['mass']
Ixx       = dataset['Ixx']
Iyy       = dataset['Iyy']
Izz       = dataset['Izz']

k_x       = dataset['k_x']
k_y       = dataset['k_y']
k_z       = dataset['k_z']
k_omega   = dataset['k_omega']
k_h       = dataset['k_h']
k_p       = dataset['k_p']
k_pv      = dataset['k_pv']
k_q       = dataset['k_q']
k_qv      = dataset['k_qv']
k_r1      = dataset['k_r1']
k_r2      = dataset['k_r2']
k_rr      = dataset['k_rr']

omega_max = dataset['omega_max']
omega_min = dataset['omega_min']
tau       = dataset['tau']

print(omega_min, omega_max)
print(dataset['dx'].shape)
print(tau)

print(g, Ixx, Iyy, Izz, k_x, k_y, k_z, k_omega, k_h, k_p, k_pv, k_q, k_qv, k_r1, k_r2, k_rr, tau, sep=',')

In [ ]:
Mx, My, Mz = 0, 0, 0

In [ ]:
def dynamics(t, y, u):    
    # state
    dx, dy, dz, vx, vy, vz, phi, theta, psi, p, q, r, omega1, omega2, omega3, omega4 = y
    
    # control input
    u1, u2, u3, u4 = u
    
    d_dx    = -q*dz + r*dy - vx
    d_dy    =  p*dz - r*dx - vy
    d_dz    = -p*dy + q*dx - vz
    
    omegas = omega1 + omega2 + omega3 + omega4
    omegas2 = omega1**2 + omega2**2 + omega3**2 + omega4**2
    
    d_vx    = -q*vz + r*vy - g*sin(theta) - k_x*omegas*vx
    d_vy    =  p*vz - r*vx + g*cos(theta)*sin(phi) - k_y*omegas*vy
    d_vz    = -p*vy + q*vx + g*cos(theta)*cos(phi) - k_z*omegas*vz - k_omega*omegas2 - k_h*(vx**2+vy**2)
    
    d_phi   = p + q*sin(phi)*tan(theta) + r*cos(phi)*tan(theta)
    d_theta = q*cos(phi) - r*sin(phi)
    d_psi   = q*sin(phi)/cos(theta) + r*cos(phi)/cos(theta)
    
    d_omega1 = (omega_min + u1*(omega_max - omega_min)-omega1)/tau
    d_omega2 = (omega_min + u2*(omega_max - omega_min)-omega2)/tau
    d_omega3 = (omega_min + u3*(omega_max - omega_min)-omega3)/tau
    d_omega4 = (omega_min + u4*(omega_max - omega_min)-omega4)/tau
    
    taux = k_p*(omega1**2-omega2**2-omega3**2+omega4**2) + k_pv*vy + Mx
    tauy = k_q*(omega1**2+omega2**2-omega3**2-omega4**2) + k_qv*vx + My
    tauz = k_r1*(-omega1+omega2-omega3+omega4) + k_r2*(-d_omega1+d_omega2-d_omega3+d_omega4) - k_rr*r + Mz
    
    d_p     = (q*r*(Iyy-Izz) + taux)/Ixx
    d_q     = (p*r*(Izz-Ixx) + tauy)/Iyy
    d_r     = (p*q*(Ixx-Iyy) + tauz)/Izz
    
    return np.array([d_dx, d_dy, d_dz, d_vx, d_vy, d_vz, d_phi, d_theta, d_psi, d_p, d_q, d_r, d_omega1, d_omega2, d_omega3, d_omega4]) 

In [ ]:
##### longer time
t = np.linspace(0,20,20000)

y_0= np.zeros(16)
y_0[12:16] = (omega_max+omega_min)/2
y_0[0] = 4.
y_0[1] = 0.
y_0[8] = 0.*np.pi/2

def dydt(t, y):
    dx, dy, dz, vx, vy, vz, phi, theta, psi, p, q, r, omega1, omega2, omega3, omega4 = y
    u = neural_network(y)
    return dynamics(t, y, u)

sol = integrate.solve_ivp(dydt, (0, 20), y_0, t_eval=t)

y_sim = sol.y.T
y_sim_w = np.array([transform_back(yi) for yi in y_sim])

x, y, z, vx, vy, vz, phi, theta, psi, p, q, r, omega1, omega2, omega3, omega4 = y_sim_w.T
u = np.array([neural_network(yi) for yi in y_sim])

In [ ]:
animation.animate(t,x,y,z-1,phi,theta,psi,u, waypoints=[np.array([0.,0.,-1.]), np.array([-4.,0.,-1.])])

# Flying a continuous trajectory

In [ ]:
from quadcopter_animation import animation
import scipy


target1 = np.array([-2.,-2.,-1.])
target2 = np.array([ 2.,-2.,-1.])
target3 = np.array([ 2., 2.,-1.])
target4 = np.array([-2., 2.,-1.])

target = target2

t = np.linspace(0, 10, 1000)
t0 = 0

y0_w = np.zeros(16)
y0_w[0:3] = target1
y0_w[12:16] = (dataset["omega_max"] + dataset["omega_min"])/2

y0 = transform(y0_w)

psi_ref = 0
u_list = []
target_list = []


def dydt(t, y):
    global target, psi_ref, u_list, t0
    # state in world coordinates
    y_w = transform_back(y)
    
    delta_pos = y_w[0:3] - target
    if target is target2:
        psi_ref = 0
        normal = np.array([np.cos(np.pi*1/4), np.sin(np.pi*1/4), 0.])
        if np.linalg.norm(delta_pos) < 0.8:
            target = target3
    elif target is target3:
        psi_ref = np.pi/2
        normal = np.array([np.cos(np.pi*3/4), np.sin(np.pi*3/4), 0.])
        if np.linalg.norm(delta_pos) < 0.8:
            target = target4
    elif target is target4:
        psi_ref = np.pi
        normal = np.array([np.cos(np.pi*5/4), np.sin(np.pi*5/4), 0.])
        if np.linalg.norm(delta_pos) < 0.8:
            target = target1
    elif target is target1:
        psi_ref = -np.pi/2
        normal = np.array([np.cos(np.pi*7/4), np.sin(np.pi*7/4), 0.])
        if np.linalg.norm(delta_pos) < 0.8:
            target = target2
    
           
    
    # describe state relative to target
    y_w[0:3] -= target
    
    nn_input = transform(y_w)
    
    nn_input[8] -= psi_ref
    
    while nn_input[8] > np.pi:
        nn_input[8] -= np.pi*2
    while nn_input[8] < -np.pi:
        nn_input[8] += np.pi*2
    
    
    # control input
    u = neural_network(nn_input) #,Mx,My,Mz)
    u_list.append((t, u))
    target_list.append((t, target))
    
    d_y = dynamics(t, y, u)
    return d_y
    

sol = integrate.solve_ivp(dydt, (t[0], t[-1]), y0, t_eval=t)
y_sim = sol.y.T
y_sim_w = np.array([transform_back(yi) for yi in y_sim])

x, y, z, vx, vy, vz, phi, theta, psi = y_sim_w[:, 0:9].T

t_ = np.array([ti for ti, ui in u_list])
u_ = np.array([ui for ti, ui in u_list])
u_ = scipy.interpolate.interp1d(t_, u_, axis=0, fill_value="extrapolate")
u_ = u_(t)

t_ = np.array([ti for ti, targeti in target_list])
target_ = np.array([targeti for ti, targeti in target_list])
target_ = scipy.interpolate.interp1d(t_, target_, axis=0, fill_value="extrapolate")
target_ = target_(t)

animation.animate(t,x,y,z,phi,theta,psi,u_, target=target_, waypoints=[target1,target2,target3,target4])

In [ ]:
import matplotlib.cm as cm
import matplotlib as mpl

norm = mpl.colors.Normalize(vmin=0, vmax=6)
cmap = cm.jet

def color_plot(x_axis, y_axis, color_axis):
    step = 10
    for i in reversed(range(step,len(x_axis),step)):
        ax = plt.gca()
        ax.plot([x_axis[i-step], x_axis[i]], [y_axis[i-step], y_axis[i]], color=cmap(norm(color_axis[i])))

In [ ]:
color_plot(y,x,V)
plt.xlabel('y [m]')
plt.ylabel('x [m]')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.gca().set_aspect('equal')
plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), label='V [m/s]')
plt.title('Simulation')
plt.show()